In [30]:
%pip install cupy-cuda12x

DEPRECATION: Loading egg at /home/ika1/.conda/envs/py311/lib/python3.11/site-packages/nms-0.0.0-py3.11-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [31]:
from hill_climbing import Climber
import cupy as cp
import pandas as pd 
import numpy as np


oof_cb = pd.read_csv("oofs/catboost_oofs.csv")
oof_curve = pd.read_csv("oofs/curve_model_oofs.csv")
oof_xgb_dumb = pd.read_csv("oofs/xgb_dumb_oofs.csv")
oof_xgb= pd.read_csv("oofs/xgb_oofs.csv")

labels = np.log1p(pd.read_csv("train.csv")["Calories"])

test_preds_curve = pd.read_csv("test_preds/curved.csv")#,Heart_Rate_Pred,Body_Temp_Pred,Duration_Pred
test_preds_gbdt = pd.read_csv("test_preds/gbdt.csv")#,xgb_best,xgb_dumb,cb

test_preds = pd.DataFrame()

#test_preds["curve_duration"] = np.log1p(test_preds_curve["Duration_Pred"])
#test_preds["curve_heartrate"] = np.log1p(test_preds_curve["Heart_Rate_Pred"])
#test_preds["curve_bodytemp"] = np.log1p(test_preds_curve["Body_Temp_Pred"])

test_preds["cb"] = np.log1p(test_preds_gbdt["cb"])
test_preds["xgb"] = np.log1p(test_preds_gbdt["xgb_best"])  
test_preds["xgb_dumb"] = np.log1p(test_preds_gbdt["xgb_dumb"])

oofs = pd.DataFrame()

oofs["cb"] = np.log1p(oof_cb["cb"])
oofs["xgb"] = np.log1p(oof_xgb["xgb"])
oofs["xgb_dumb"] = np.log1p(oof_xgb_dumb["xgb_dumb"])
#oofs["curve_duration"] = np.log1p(oof_curve["Duration_Pred"])
#oofs["curve_heartrate"] = np.log1p(oof_curve["Heart_Rate_Pred"])
#oofs["curve_bodytemp"] = np.log1p(oof_curve["Body_Temp_Pred"])


def rmse(y_true, y_pred):
    if isinstance(y_true, cp.ndarray):  # GPU mode
        return cp.sqrt(cp.mean((y_true - y_pred) ** 2))
    else:  # CPU mode
        return np.sqrt(np.mean((y_true - y_pred) ** 2))

climber = Climber(
    objective="minimize",
    eval_metric=rmse,
    allow_negative_weights=False,    
    score_decimal_places=6,
    precision=0.001,
    n_jobs=1,
    use_gpu=True
).fit(oofs, labels)



Configuration

   Metric:                       rmse
   Objective:                    minimize
   Precision:                    0.001
   Allow negative weights:       False
   Starting model:               best
   Number of parallel jobs:      1
   Number of models:             3
   Using GPU:                    False


Models

   xgb        0.050652 █ (best)
   xgb_dumb   0.052870 ███████████████████
   cb         0.055240 ████████████████████████████████████████


Running Hill Climbing

   Iter   Model        Weight        Score      Improvement         Time
   ─────────────────────────────────────────────────────────────────────
      0   xgb           1.000     0.050652                -            -


Results

   Number of models in ensemble:      1
   Total time:                        6.38 seconds
   Average iteration time:            0.02 seconds
   Final score:                       0.050652


In [32]:
preds = np.expm1(climber.predict(test_preds))

In [33]:
test_df=pd.read_csv("test.csv")

submission = pd.DataFrame({"id": test_df["id"], "prediction": np.expm1(preds)})

submission.to_csv("submission_hillclimb.csv", index=False)


In [35]:
import numpy as np
from scipy.optimize import minimize

# 1. Prepare your OOF predictions and labels
X = oofs.values  # Shape (n_samples, n_models)
y = labels.values  # True target values

# 2. Define objective function
def objective(weights):
    """Calculate RMSE for given ensemble weights"""
    weighted_avg = X @ weights  # Matrix multiplication
    return np.sqrt(np.mean((y - weighted_avg) ** 2))

# 3. Set up constraints
constraints = (
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}  # Weights sum to 1
)

# 4. Set bounds (force non-negative weights)
bounds = [(0, 1) for _ in range(X.shape[1])]

# 5. Initial guess (equal weights)
initial_weights = np.ones(X.shape[1]) / X.shape[1]

# 6. Run optimization
result = minimize(
    objective,
    x0=initial_weights,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints,
    options={'maxiter': 1000, 'ftol': 1e-8}
)

# 7. Extract optimal weights
best_weights = result.x
print("Optimal weights:", dict(zip(oofs.columns, best_weights)))

# 8. Apply to test data (MUST match column order!)
test_matrix = test_preds[oofs.columns].values
test_pred = test_matrix @ best_weights

# 9. Convert back from log scale
final_preds = np.expm1(test_pred)

Optimal weights: {'cb': 0.0, 'xgb': 1.0, 'xgb_dumb': 5.551115123125783e-17}
